# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))   
    #print(file_path_list)    

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# check the first few lines of the data
print(full_data_rows_list[0:5])
print(len(full_data_rows_list))

[['Muse', 'Logged In', 'Harper', 'M', '1', 'Barrett', '209.50159', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54069E+12', '275', 'Supermassive Black Hole (Twilight Soundtrack Version)', '200', '1.54172E+12', '42'], ['Beastie Boys', 'Logged In', 'Harper', 'M', '2', 'Barrett', '161.56689', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54069E+12', '275', 'Lighten Up', '200', '1.54172E+12', '42'], ['Shakira', 'Logged In', 'Harper', 'M', '3', 'Barrett', '145.84118', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54069E+12', '275', 'Pienso En Ti', '200', '1.54172E+12', '42'], ['Selena', 'Logged In', 'Harper', 'M', '4', 'Barrett', '172.66893', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54069E+12', '275', 'Amor Prohibido', '200', '1.54172E+12', '42'], ['Kid Cudi Vs Crookers', 'Logged In', 'Harper', 'M', '5', 'Barrett', '162.97751', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'Ne

In [4]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modeling with Apache Cassandra

#### Work with the CSV file titled <font color=red>event_datafile_new.csv</font>, which contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### Creating a Cluster

In [6]:
# This makes a connection to a Cassandra instance on the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}""")

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create three queries on the data

### Query 1. Provide the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Query: SELECT artist, song, length FROM music_app_history WHERE sessionId = 338 AND itemInSession = 4

Description: the Primary Key has two fields. sessionId is the partition key, and itemInSession is the clustering key. Partitioning is done by sessionId and within that partition, rows are ordered by the itemInSession.

In [9]:
# create table
query = "CREATE TABLE IF NOT EXISTS music_app_history"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)    

In [10]:
# insert data
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [11]:
# run query
query = "SELECT artist, song, length FROM music_app_history WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)    
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### Query 2. Provide only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Query: SELECT * FROM music_app_history1 WHERE userId = 10 AND sessionId = 182

Description: the Primary Key has two fields. userId and sessionId are the partition keys, and itemInSession is the clustering key. Partitioning is done by userId and sessionId and within that partition, rows are ordered by the itemInSession.

In [12]:
# create table
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [13]:
# insert data
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])) 

In [14]:
# run query
query = "SELECT artist, song, firstname, lastname FROM song_playlist_session WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


### Query 3. Provide every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Query: SELECT * FROM music_app_history2 WHERE song = 'All Hands Against His Own'

Description: the Primary Key has two fields. song is the partition key, and userId is the clustering key. Partitioning is done by song and within that partition, rows are ordered by the userId.

In [15]:
# create table
query = "CREATE TABLE IF NOT EXISTS user_listening_history"
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)                      

In [16]:
# insert data
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_listening_history (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [17]:
# run query
query = "SELECT firstname, lastname FROM user_listening_history WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)   
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [18]:
query = 'drop table music_app_history'
try: 
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
query = 'drop table song_playlist_session'
try: 
    session.execute(query)
except Exception as e:
    print(e)

In [20]:
query = 'drop table user_listening_history'
try: 
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()